In [ ]:
!unzip '/content/drive/MyDrive/data/CAP Dataset.zip'

In [ ]:
# !unzip '/content/drive/MyDrive/data/Copy of images_normalized.zip'

In [ ]:
import os
import tensorflow as tf
print(len(tf.config.list_physical_devices('GPU')))

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import Sequential
import os

In [ ]:
train_dir = '/content/dataset/train/'
test_dir = '/content/dataset/test/'

In [ ]:
datagen = ImageDataGenerator(rescale = 1/255. ,
                              validation_split=0.2,
                             )
datagen2 = ImageDataGenerator(rescale = 1/255.
                              )
train_data = datagen.flow_from_directory(
    train_dir,
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary',
    color_mode='rgb',
    shuffle=True,
    subset = 'training'
)

valid_data = datagen.flow_from_directory(
    train_dir,
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary',
    color_mode='rgb',
    shuffle=True,
    subset = 'validation'

)




In [ ]:
# train_data2 = tf.data.Dataset.from_generator(
#     lambda: train_data,
#     output_signature=(
#         tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
#         tf.TensorSpec(shape=(None,), dtype=tf.float32)
#     )
# )
# valid_data2 = tf.data.Dataset.from_generator(
#     lambda: valid_data,
#     output_signature=(
#         tf.TensorSpec(shape=(None,224, 224, 3), dtype=tf.float32),
#         tf.TensorSpec(shape=(None,), dtype=tf.float32)
#     )
# )

In [ ]:
test_data = datagen2.flow_from_directory(
    test_dir,
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary',
    color_mode='rgb'
)


In [ ]:
# input_layer = tf.keras.Input(shape=(32, 32, 3))

# x = tf.keras.layers.Conv2D(16, 3, activation='relu', padding='same')(input_layer)
# x = tf.keras.layers.Conv2D(16, 3, activation='relu', padding='same')(x)
# x = tf.keras.layers.MaxPool2D()(x)

# x = tf.keras.layers.SeparableConv2D(32, 3, activation='relu', padding='same')(x)
# x = tf.keras.layers.SeparableConv2D(32, 3, activation='relu', padding='same')(x)
# x = tf.keras.layers.BatchNormalization()(x)
# x = tf.keras.layers.MaxPool2D()(x)

# x = tf.keras.layers.SeparableConv2D(64, 3, activation='relu', padding='same')(x)
# x = tf.keras.layers.SeparableConv2D(64, 3, activation='relu', padding='same')(x)
# x = tf.keras.layers.BatchNormalization()(x)
# x = tf.keras.layers.MaxPool2D()(x)

# x = tf.keras.layers.SeparableConv2D(128, 3, activation='relu', padding='same')(x)
# x = tf.keras.layers.SeparableConv2D(128, 3, activation='relu', padding='same')(x)
# x = tf.keras.layers.BatchNormalization()(x)
# x = tf.keras.layers.MaxPool2D()(x)
# x = tf.keras.layers.Dropout(0.2)(x)

# x = tf.keras.layers.SeparableConv2D(256, 3, activation='relu', padding='same')(x)
# x = tf.keras.layers.SeparableConv2D(256, 3, activation='relu', padding='same')(x)
# x = tf.keras.layers.BatchNormalization()(x)
# x = tf.keras.layers.MaxPool2D()(x)
# x = tf.keras.layers.Dropout(0.2)(x)

# x = tf.keras.layers.Flatten()(x)

# x = tf.keras.layers.Dense(512, activation='relu')(x)
# x = tf.keras.layers.BatchNormalization()(x)
# x = tf.keras.layers.Dropout(0.7)(x)

# x = tf.keras.layers.Dense(128, activation='relu')(x)
# x = tf.keras.layers.BatchNormalization()(x)
# x = tf.keras.layers.Dropout(0.5)(x)

# x = tf.keras.layers.Dense(64, activation='relu')(x)
# x = tf.keras.layers.BatchNormalization()(x)
# x = tf.keras.layers.Dropout(0.3)(x)

# output_layer = tf.keras.layers.Dense(1, activation='sigmoid')(x)

# model = tf.keras.Model(inputs=input_layer, outputs=output_layer)

# model.summary()

In [ ]:
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics= ['accuracy'])

In [ ]:
# len(train_data)

In [ ]:
# history1 = model.fit(
#     train_data2,
#     steps_per_epoch = len(train_data),
#     validation_data = valid_data2,
#     validation_steps = int(0.25* len(valid_data)),
#     epochs=50
# )


## Using VGG model

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Model

base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
METRICS = [
    'accuracy',
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]
for layer in base_model.layers:
    layer.trainable = False

x = Flatten()(base_model.output)
x = Dense(128, activation='relu')(x)
# x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)

vggmodel = Model(inputs=base_model.input, outputs=output)

vggmodel.summary()


In [ ]:
from tensorflow.keras.optimizers import Adam

optimizer = Adam(lr = 1e-4)

In [ ]:
vggmodel.compile(optimizer=optimizer, loss='binary_crossentropy', metrics= ['accuracy'])

In [ ]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
ckpt = ModelCheckpoint('./best.h5',
                       monitor= "val_loss",
                       save_best_only=True

                      )

history_1 = vggmodel.fit(
    train_data2,
    steps_per_epoch = len(train_data),
    validation_data = valid_data2,
    validation_steps = int(0.25* len(valid_data)),
    epochs=50,
    callbacks = [early_stopping_cb ]
)

vggmodel.save('./modelvgg_epoch.h5')

In [ ]:
import matplotlib.pyplot as plt

def plot_loss_curves(history):

  loss = history.history['loss']
  val_loss = history.history['val_loss']

  accuracy = history.history['accuracy']
  val_accuracy = history.history['val_accuracy']

  epochs = range(len(history.history['loss']))

  plt.plot(epochs, loss, label='training_loss')
  plt.plot(epochs, val_loss, label='val_loss')
  plt.title('Loss')
  plt.xlabel('Epochs')
  plt.legend()

  plt.figure()
  plt.plot(epochs, accuracy, label='training_accuracy')
  plt.plot(epochs, val_accuracy, label='val_accuracy')
  plt.title('Accuracy')
  plt.xlabel('Epochs')
  plt.legend();


In [ ]:
plot_loss_curves(history_1)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Resnet

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy

In [ ]:
train_dir = '/content/dataset/train/'
test_dir = '/content/dataset/test/'

In [ ]:
train_data = keras.preprocessing.image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=(224, 224),
    batch_size=62,
    label_mode="binary",
)

test_data = keras.preprocessing.image_dataset_from_directory(
    test_dir,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=(224, 224),
    batch_size=64 ,
    label_mode="binary",
)


In [ ]:

import torchvision
pretrained_resnet_weights = torchvision.models.ResNet50_Weights.IMAGENET1K_V1

pretrained_resnet = tf.keras.applications.resnet50.ResNet50(
    weights='imagenet',
    input_shape=(224, 224, 3),  # Adjust input shape to your dataset
    include_top=False,
)


In [ ]:
for layer in pretrained_.layers:
    layer.trainable = False

class_names = ['A', 'not_A']


In [ ]:
pretrained_vit.output.shape[1]

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras import Model
from tensorflow.keras.applications import ResNet50

# Define data preprocessing and augmentation
random = tf.keras.Sequential([
    tf.keras.layers.Rescaling(scale=1./255),
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.Resizing(224, 224),
])

# Load a pretrained ResNet model (you can use any pretrained model)
pretrained_resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers in the pretrained ResNet model
for layer in pretrained_resnet.layers:
    layer.trainable = False

# Create your custom head for the model
x = Flatten()(pretrained_resnet.output)
x = Dense(128, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

# Combine the random preprocessing and your custom head
resnetmodel = Model(inputs=random.input, outputs=output)

# Print a summary of the model
resnetmodel.summary()


In [ ]:
from tensorflow.keras.optimizers import Adam
optimizer = Adam(lr = 1e-4)
resnetmodel.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Training loop
num_epochs = 30

history = resnetmodel.fit(
    train_data,
    epochs=num_epochs,
    validation_data=test_data,
)



# Using VIT

In [ ]:
import tensorflow as tf


In [ ]:
!unzip '/content/drive/MyDrive/data/CAP Dataset.zip'

In [ ]:
train_dir = '/content/dataset/train/'
test_dir = '/content/dataset/test/'

In [ ]:
import tensorflow as tf

# Load the image
image_path = "/content/data/train/A/cwtm_image_1.png"
image = tf.image.decode_image(tf.io.read_file(image_path))

# Get the shape of the image
shape = tf.shape(image)

# Extract the dimensions (shape)
height = shape[0]
width = shape[1]
channels = shape[2]

# Print the dimensions
print(f"Image width: {width} pixels")
print(f"Image height: {height} pixels")
print(f"Number of channels: {channels}")


In [ ]:
import tensorflow as tf

# Load the image
image_path = "your_image.jpg"
image = tf.image.decode_image(tf.io.read_file(image_path))

# Get the shape of the image
shape = tf.shape(image)

# Extract the dimensions (shape)
height = shape[0]
width = shape[1]
channels = shape[2]

# Print the dimensions
print(f"Image width: {width} pixels")
print(f"Image height: {height} pixels")
print(f"Number of channels: {channels}")


In [ ]:
train_data = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    label_mode = 'binary',
    image_size=(224,224) ,
    batch_size =32,
    color_mode = 'rgb'
)


test_data = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    label_mode = 'binary',
    image_size=(224,224) ,
    batch_size =32,
    color_mode = 'rgb'
)

In [ ]:
print(tf.__version__)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

In [ ]:
transform = keras.Sequential([
    layers.Rescaling(scale=1./255),
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.1),
    layers.Resizing(224, 224),
])



In [ ]:
data_augmentation = keras.Sequential([
    preprocessing.RandomFlip(),
    preprocessing.RandomRotation(0.2),
    preprocessing.RandomZoom(0.2),
    preprocessing.RandomHeight(0.2),
    preprocessing.RandomWidth(0.2)
    # preprocessing.Rescale(1/255.),
])

In [ ]:
input_shape = (224,224,3)
base_model = tf.keras.applications.EfficientNetB0(include_top = False)
base_model.trainable = True

for layer in base_model.layers[:-10]:
  layer.trainable  = False


inputs = layers.Input(shape = input_shape)

x = data_augmentation(inputs)
x = base_model(x , training = False)

x = layers.GlobalAveragePooling2D()(x)

outputs = layers.Dense(1 , activation='sigmoid')(x)



model_1 = keras.Model(inputs , outputs)

model_1.summary()


In [ ]:
import tensorflow_hub as hub
# https://tfhub.dev/sayakpaul/vit_s16_classification/1



input_shape = (224,224,3)



base_model = hub.KerasLayer("https://tfhub.dev/sayakpaul/vit_s16_classification/1" , trainable = False , input_shape = input_shape)




In [ ]:
model_1 = tf.keras.Sequential([
    base_model,
    tf.keras.layers.Dense(1 , activation='sigmoid')
])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


path = '/content/'
model_ckpt= tf.keras.callbacks.ModelCheckpoint(filepath = path,
                                               save_weight_only = True,
                                               save_best_only = False,
                                               save_freq = 'epoch',
                                               verbose=1
                                               )

In [ ]:
model_1.compile(loss = 'binary_crossentropy', optimizer= tf.keras.optimizers.Adam(lr=1e-4) ,metrics = ['accuracy'])


history_1 = model_1.fit(train_data ,
                        epochs  = 30,
                        steps_per_epoch= len(train_data),
                        validation_data = test_data,
                        validation_steps = int(0.15*len(test_data)),
                        callbacks = [early_stopping_cb , model_ckpt]

                        )

In [ ]:
model_1.evaluate(test_data)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
import tensorflow_hub as hub

# Define the input shape
input_shape = (224, 224, 3)  # You can adjust the input shape to match your dataset

# Load the ViT-B/16 model from TensorFlow Hub
model_url = "https://tfhub.dev/sayakpaul/vit_b16_classification/1"
vit_model = hub.load(model_url)

# Create a custom Keras layer for the ViT model
class VitLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(VitLayer, self).__init__()

    def call(self, inputs):
        return vit_model(inputs)

# Create a new model with a custom output layer for your specific task
inputs = Input(shape=input_shape)
vit_output = VitLayer()(inputs)
# Add a spatial dimension for GlobalAveragePooling2D
vit_output = tf.expand_dims(vit_output, axis=1)
vit_output = tf.expand_dims(vit_output, axis=1)
vit_layer = tf.keras.layers.GlobalAveragePooling2D()(vit_output)
output = Dense(1, activation='sigmoid')(vit_layer)

model = tf.keras.Model(inputs=inputs, outputs=output)

# Compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Print a summary of the model
model.summary()


In [ ]:
history = model.fit(train_data ,
                        epochs  = 30,
                        steps_per_epoch= len(train_data),
                        validation_data = test_data,
                        validation_steps = int(0.15*len(test_data)),
                        # callbacks = [ model_ckpt]

                        )

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout
import tensorflow_hub as hub

input_shape = (224, 224, 3)  

model_url = "https://tfhub.dev/sayakpaul/vit_b16_classification/1"
vit_model = hub.load(model_url)

inputs = Input(shape=input_shape)
vit_output = vit_model(inputs)
output = Dense(1, activation='sigmoid')(vit_output)
model = tf.keras.Model(inputs=inputs, outputs=output)

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',  # Use the appropriate loss function for your task
    metrics=['accuracy']
)

# Print a summary of the model
model.summary()


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)


path = '/content/'
model_ckpt= tf.keras.callbacks.ModelCheckpoint(filepath = path,
                                               save_weight_only = True,
                                               save_best_only = False,
                                               save_freq = 'epoch',
                                               verbose=1
                                               )

In [ ]:
def lr_scheduler(epoch, lr):
    if epoch % 10 == 0 and epoch > 0:
        return lr * 0.5
    return lr

learning_rate_scheduler = keras.callbacks.LearningRateScheduler(lr_scheduler)


In [ ]:
model_2.compile(loss = 'binary_crossentropy', optimizer= keras.optimizers.Adam(lr=1e-4) ,metrics = ['accuracy'])


history_2 = model_2.fit(train_data ,
                        epochs  = 30,
                        steps_per_epoch= len(train_data),
                        validation_data = test_data,
                        validation_steps = int(0.15*len(test_data)),
                        callbacks = [early_stopping_cb , model_ckpt , learning_rate_scheduler]

                        )

## EfficientNet

In [ ]:
import tensorflow as tf

# Define your data augmentation layer
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.2),
    tf.keras.layers.experimental.preprocessing.RandomTranslation(0.1, 0.1),
])

# You can then apply this layer to your input data


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load a pre-trained EfficientNet model
model_url = "https://tfhub.dev/google/efficientnet/b3/classification/1"
efficientnet = hub.load(model_url)

# Modify the model for binary classification
model = keras.Sequential([
    efficientnet,
    layers.GlobalAveragePooling2D(),  # Global Average Pooling layer
    layers.Dense(1, activation='sigmoid')
])

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load a pre-trained EfficientNet model
model_url = "https://tfhub.dev/google/efficientnet/b3/classification/1"
efficientnet = hub.load(model_url)

# Define your data augmentation settings
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load your image datasets
train_data = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    label_mode='binary',
    image_size=(224, 224),
    batch_size=32,
    color_mode='rgb'
)

test_data = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    label_mode='binary',
    image_size=(224, 224),
    batch_size=32,
    color_mode='rgb'
)

# Create your model
model = keras.Sequential([
    efficientnet,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=keras.optimizers.Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Apply data augmentation using the ImageDataGenerator
train_data = train_data.map(lambda x, y: (datagen.random_transform(x), y))

# Train the model
model.fit(train_data, validation_data=test_data, epochs=10)


# Using GASF images

In [ ]:
!unzip '/content/drive/MyDrive/data/data.zip'

In [ ]:
import tensorflow as tf
train_dir = '/content/data/train'
test_dir = '/content/data/test'

In [ ]:
# import tensorflow as tf

# # Load the image
# image_path = "/content/data/train/A/cwtm_image_1.png"
# image = tf.image.decode_image(tf.io.read_file(image_path))

# # Get the shape of the image
# shape = tf.shape(image)

# # Extract the dimensions (shape)
# height = shape[0]
# width = shape[1]
# channels = shape[2]

# # Print the dimensions
# print(f"Image width: {width} pixels")
# print(f"Image height: {height} pixels")
# print(f"Number of channels: {channels}")


In [ ]:
train_data = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    label_mode = 'binary',
    image_size=(224,224) ,
    batch_size =32,
    color_mode = 'rgb'
)


test_data = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    label_mode = 'binary',
    image_size=(224,224) ,
    batch_size =32,
    color_mode = 'rgb'
)

In [ ]:
from tensorflow.keras.layers.experimental import preprocessing
data_augmentation = tf.keras.Sequential([
    preprocessing.RandomFlip(),
    preprocessing.RandomRotation(0.2),
    preprocessing.RandomZoom(0.2),
    preprocessing.RandomHeight(0.2),
    preprocessing.RandomWidth(0.2)
    # preprocessing.Rescale(1/255.),
])

In [ ]:
from tensorflow.keras import layers
input_shape = (224,224,3)
base_model = tf.keras.applications.EfficientNetB0(include_top = False)
base_model.trainable = True

for layer in base_model.layers[:-10]:
  layer.trainable  = False


inputs = layers.Input(shape = input_shape)

x = data_augmentation(inputs)
x = base_model(x , training = False)

x = layers.GlobalAveragePooling2D()(x)

outputs = layers.Dense(1 , activation='sigmoid')(x)



model_1 = tf.keras.Model(inputs , outputs)

model_1.summary()


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


path = '/content/'
model_ckpt= tf.keras.callbacks.ModelCheckpoint(filepath = path,
                                               save_weight_only = True,
                                               save_best_only = True,
                                               save_freq = 'epoch',
                                               verbose=1
                                               )

In [ ]:
model_1.compile(loss = 'binary_crossentropy', optimizer= tf.keras.optimizers.Adam(lr=1e-4) ,metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

history_1 = model_1.fit(train_data ,
                        epochs  = 30,
                        steps_per_epoch= len(train_data),
                        validation_data = test_data,
                        validation_steps = int(0.15*len(test_data)),
                        callbacks = [early_stopping_cb , model_ckpt]
)

In [ ]:
model_1.evaluate(test_data)

In [ ]:
custom_threshold = 0.6  # Example threshold value
validation_data = test_data  # Replace with the actual validation dataset
y_pred = model_1.predict(validation_data)
y_pred_binary = (y_pred > custom_threshold).astype(int)

In [ ]:
label_mapping = {'A': 1, 'notA': 1}

true_labels = [label_mapping[path.split('/')[-2]] for path in file_paths]


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

accuracy = accuracy_score(true_labels, y_pred_binary)
precision = precision_score(true_labels, y_pred_binary)
recall = recall_score(true_labels, y_pred_binary)
f1 = f1_score(true_labels, y_pred_binary)
conf_matrix = confusion_matrix(true_labels, y_pred_binary)

# Print or use the evaluation results as needed
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')
print('Confusion Matrix:')
print(conf_matrix)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier  # Correct import
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Define a function to create the Keras model
def create_model(learning_rate=0.001, num_units=256):
    base_model = tf.keras.applications.EfficientNetB0(
        include_top=False, weights='imagenet', input_shape=(224, 224, 3)
    )

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(num_units, activation='relu')(x)
    predictions = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    for layer in base_model.layers:
        layer.trainable = False

    model.compile(
        optimizer=Adam(learning_rate),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model

# Specify the data directory
train_data_dir = 'path_to_training_data'  # Replace with the actual path

# Create an ImageDataGenerator to load and preprocess the images
datagen = ImageDataGenerator(rescale=1.0 / 255.0)

train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

X = train_generator  # Set X to the generator
y = train_generator.classes  # Set y to the class labels

# Define hyperparameter grid
param_grid = {
    'learning_rate': [0.001, 0.01, 0.1],
    'num_units': [128, 256, 512]
}

# Create a KerasClassifier
model = KerasClassifier(build_fn=create_model, epochs=20, batch_size=32, verbose=0)

# Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, y)

# Print the best hyperparameters and accuracy
print(f'Best Parameters: {grid_result.best_params_}')
print(f'Best Accuracy: {grid_result.best_score_}')


## Torch + Vit

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
def set_seeds(seed: int=42):
    # Set the seed for general torch operations
    torch.manual_seed(seed)
    # Set the seed for CUDA torch operations (ones that happen on the GPU)
    torch.cuda.manual_seed(seed)

In [ ]:
# 1. Get pretrained weights for ViT-Base
pretrained_vit_weights = torchvision.models.ViT_B_16_Weights.DEFAULT 

# 2. Setup a ViT model instance with pretrained weights
pretrained_vit = torchvision.models.vit_b_16(weights=pretrained_vit_weights).to(device)

# 3. Freeze the base parameters
for parameter in pretrained_vit.parameters():
    parameter.requires_grad = False
    print(parameter.shape)
    
# 4. Change the classifier head 
class_names = ['A','Not_A']

set_seeds()
pretrained_vit.heads = nn.Linear(in_features=768, out_features=len(class_names)).to(device)
# pretrained_vit

torch.Size([1, 1, 768])
torch.Size([768, 3, 16, 16])
torch.Size([768])
torch.Size([1, 197, 768])
torch.Size([768])
torch.Size([768])
torch.Size([2304, 768])
torch.Size([2304])
torch.Size([768, 768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([3072, 768])
torch.Size([3072])
torch.Size([768, 3072])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([2304, 768])
torch.Size([2304])
torch.Size([768, 768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([3072, 768])
torch.Size([3072])
torch.Size([768, 3072])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([2304, 768])
torch.Size([2304])
torch.Size([768, 768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([3072, 768])
torch.Size([3072])
torch.Size([768, 3072])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([2304, 768])
torch.Size([2304])
torch.Size([768, 768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([3072, 768])


In [ ]:
from torchinfo import summary

# Print a summary using torchinfo (uncomment for actual output)
summary(model=pretrained_vit, 
        input_size=(64, 3, 224, 224), # (batch_size, color_channels, height, width)
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
VisionTransformer (VisionTransformer)                        [64, 3, 224, 224]    [64, 2]              768                  Partial
├─Conv2d (conv_proj)                                         [64, 3, 224, 224]    [64, 768, 14, 14]    (590,592)            False
├─Encoder (encoder)                                          [64, 197, 768]       [64, 197, 768]       151,296              False
│    └─Dropout (dropout)                                     [64, 197, 768]       [64, 197, 768]       --                   --
│    └─Sequential (layers)                                   [64, 197, 768]       [64, 197, 768]       --                   False
│    │    └─EncoderBlock (encoder_layer_0)                   [64, 197, 768]       [64, 197, 768]       (7,087,872)          False
│    │    └─EncoderBlock (encoder_layer_1)                   [64, 197, 768]       [64, 

In [ ]:
# Setup directory paths to train and test images
train_dir = 'C:/Users/Machine Learning GPU/114/Balanced_dataset/balanced_train'
test_dir = 'C:/Users/Machine Learning GPU/114/Balanced_dataset/balanced_test'

In [ ]:
# Get automatic transforms from pretrained ViT weights
pretrained_vit_transforms = pretrained_vit_weights.transforms()
print(pretrained_vit_transforms)

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)


In [ ]:
import os

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloaders(
    train_dir: str, 
    test_dir: str, 
    transform: transforms.Compose, 
    batch_size: int, 
    num_workers: int=NUM_WORKERS
):

    # Use ImageFolder to create dataset(s)
    train_data = datasets.ImageFolder(train_dir, transform=transform)
    test_data = datasets.ImageFolder(test_dir, transform=transform)

    # Get class names
    class_names = train_data.classes

    # Turn images into data loaders
    train_dataloader = DataLoader(
      train_data,
      batch_size=batch_size,
      shuffle=True,
      num_workers=num_workers,
      pin_memory=True,
    )
    test_dataloader = DataLoader(
      test_data,
      batch_size=batch_size,
      shuffle=False,
      num_workers=num_workers,
      pin_memory=True,
    )

    return train_dataloader, test_dataloader, class_names

In [ ]:
# Setup dataloaders
train_dataloader_pretrained, test_dataloader_pretrained, class_names = create_dataloaders(train_dir=train_dir,
                                                                                                     test_dir=test_dir,
                                                                                                     transform=pretrained_vit_transforms,
                                                                                                     batch_size=64)

In [ ]:
from tqdm.auto import tqdm
from typing import Dict, List, Tuple

def train_step(model: torch.nn.Module, 
               dataloader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, 
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
    model.train()
    train_loss, train_acc = 0, 0
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # 1. Forward pass
        y_pred = model(X)

        loss = loss_fn(y_pred, y)
        train_loss += loss.item() 

        #Optimizer zero grad
        optimizer.zero_grad()
        loss.backward()
        
        #Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        #Optimizer step
        optimizer.step()

        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred)

    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

def test_step(model: torch.nn.Module, 
              dataloader: torch.utils.data.DataLoader, 
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
    model.eval() 
    test_loss, test_acc = 0, 0

    with torch.inference_mode():
        for batch, (X, y) in enumerate(dataloader):
            
            X, y = X.to(device), y.to(device)
            test_pred_logits = model(X)
            
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()

            test_pred_labels = test_pred_logits.argmax(dim=1)
            test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc

def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List]:
    
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }
    
    model.to(device)

    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
        test_loss, test_acc = test_step(model=model,
                                        dataloader=test_dataloader,
                                        loss_fn=loss_fn,
                                        device=device)
        
        scheduler.step()

        # Print out what's happening
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    return results


In [ ]:
from torch.optim.lr_scheduler import StepLR

fine_tune_layers = pretrained_vit.heads.parameters()

# Create optimizer and loss function
optimizer = torch.optim.Adam(params=[{'params': fine_tune_layers}], lr=1e-4)
loss_fn = torch.nn.CrossEntropyLoss()

# Create a learning rate scheduler
scheduler = StepLR(optimizer, step_size=10, gamma=0.5)

# Train the classifier head of the pretrained ViT feature extractor model
set_seeds()
pretrained_vit_results = train(model=pretrained_vit,
                                      train_dataloader=train_dataloader_pretrained,
                                      test_dataloader=test_dataloader_pretrained,
                                      optimizer=optimizer,
                                      loss_fn=loss_fn,
                                      epochs=30,
                                      device=device)

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.5929 | train_acc: 0.6870 | test_loss: 0.5696 | test_acc: 0.7103
Epoch: 2 | train_loss: 0.5589 | train_acc: 0.7191 | test_loss: 0.5562 | test_acc: 0.7218
Epoch: 3 | train_loss: 0.5496 | train_acc: 0.7258 | test_loss: 0.5505 | test_acc: 0.7268
Epoch: 4 | train_loss: 0.5449 | train_acc: 0.7301 | test_loss: 0.5482 | test_acc: 0.7299
Epoch: 5 | train_loss: 0.5415 | train_acc: 0.7326 | test_loss: 0.5454 | test_acc: 0.7314
Epoch: 6 | train_loss: 0.5388 | train_acc: 0.7339 | test_loss: 0.5418 | test_acc: 0.7343
Epoch: 7 | train_loss: 0.5369 | train_acc: 0.7360 | test_loss: 0.5405 | test_acc: 0.7355
Epoch: 8 | train_loss: 0.5355 | train_acc: 0.7363 | test_loss: 0.5412 | test_acc: 0.7338
Epoch: 9 | train_loss: 0.5340 | train_acc: 0.7386 | test_loss: 0.5382 | test_acc: 0.7372
Epoch: 10 | train_loss: 0.5331 | train_acc: 0.7390 | test_loss: 0.5367 | test_acc: 0.7384
Epoch: 11 | train_loss: 0.5314 | train_acc: 0.7391 | test_loss: 0.5365 | test_acc: 0.7384
Epoch: 12 | train_l